<a href="https://colab.research.google.com/github/HollandKnight/Text_from_Video_Extractor/blob/master/Video_to_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Video capture a list of text entries on your iOS or Android device that you can't otherwise download or copy; convert to JPG frames; OCR the frames and programatically create spreadsheet with output.

In [0]:
# Program To Read video 
# and Extract Frames 
import cv2 

# Function to extract frames 
def FrameCapture(path): 
	# Path to video file 
    vidObj = cv2.VideoCapture(path) 
    print(type(vidObj))
	# Used as counter variable 
    count = 0
	# checks whether frames were extracted 
    success = 1
    while success:
        success, image = vidObj.read() 
        print(success)
        cv2.imwrite("frame%d.jpg" % count, image) 
        count += 1

FrameCapture('file.MP4') 

In [0]:
try:
    from pdf2image import convert_from_path
except:
    !sudo apt install poppler-utils
    !pip install pdf2image
    from pdf2image import convert_from_path
    
#!sudo apt install libtesseract-dev
#!sudo apt install tesseract-ocr
#!pip install pytesseract
import cv2
import sys
import pytesseract
import os
import shutil 
import re
import pandas as pd
rootdir = './'
output_path = './'
i=0
x = 20
entries = []
print('ok')
#for subdir, dirs, files in os.walk(rootdir):
    #for file in files:
for file in os.listdir(rootdir):
    if 1 == 1:
        i=i+1
        if file.startswith('frame'):
            print(file)
            if i >= x:
                try:
                    imPath = output_path
                    config = ('-l eng --oem 1 --psm 3')
                    im = cv2.imread(imPath, cv2.IMREAD_COLOR)
                    text = pytesseract.image_to_string(im, config=config)
                    print(text)
                    entries.append(text)
                    x = x+20
                except Exception as e: print(e)
                    
df = pd.DataFrame()
df['entries'] = entries
df.to_excel('zsafe.xlsx')

def remove_garbage(text):
    clean = re.sub('Nx\w+', '', str(text))
    clean = re.sub('NX\w+', '', str(clean))
    clean = re.sub('\n{2,}', '\n***********\n', str(clean)) #This becomes delimiter between registrants
    clean = re.sub(r'bbs', '', str(clean))
    clean = re.sub(r'bbe', '', str(clean))
    clean = re.sub('Q[\n+\s+]', '', str(clean)) 
    clean = re.sub('\|', '', str(clean))
    clean = re.sub('\s+a\s+', '', str(clean))
    print(clean)
    return clean

df['entries'] = df['entries'].apply(remove_garbage)

name = []
pos = []
co = []

for index, row in df.iterrows():
    print('test')
    segments = re.split(r'\*\*\*\*\*\*\*\*\*\*\*', df['entries'][index])
    for attendee in segments:
        line = re.split('\n', attendee)
        try:
            name_item = line[1]
            print(line[1])
            pos_item = line[2]
            print(line[2])
            co_item = line[3]
            print(line[3])
            name.append(name_item)
            print(line[1])
            pos.append(pos_item)
            print(line[2])
            co.append(co_item)
            print(line[3])
        except:
            print('err')
    
df_2 = pd.DataFrame()
df_2['name'] = name
df_2['pos'] = pos
df_2['co'] = co

display(df_2)
df_2.to_excel('test.xlsx')
